In [25]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from algorithms.astar2k import astar2k
from algorithms.thetastar import thetastar
from util.structures import Map, Node
from util.containers import Open, Closed, OpenAndClosed
from util import functions as uf
from test import movingai_util as mutil
from draw import draw
import random
%matplotlib inline

In [20]:
def simple_test(search_func, task, *args):
    '''
    simple_test runs search_func on one task (use a number from 0 to 25 to choose a certain debug task on simple map or None to choose a random task from this pool) with *args as optional arguments and displays:
     - 'Path found!' and some statistics -- path was found
     - 'Path not found!' -- path was not found
     - 'Execution error' -- an error occurred while executing the SearchFunction In first two cases function also draws visualisation of the task
    '''
    
    height = 15
    width = 30
    map_str = '''
. . . . . . . . . . . . . . . . . . . . . # # . . . . . . .  
. . . . . . . . . . . . . . . . . . . . . # # . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . . . . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . # # # # # . . . . 
. . . # # . . . . . . . . # # . . . . . . # # # # # . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . # # . . . . . . . . . . . . . . .
. . . . . . . . . . . . . # # . . . . . . . . . . . . . . .
'''

    task_map = Map()
    task_map.read_from_string(map_str, width, height)
    starts = [(9, 0), (13, 0), (7, 28), (14, 29), (4, 1), (0, 17), (5, 6), (5, 20), (12, 2), (7, 28), (11, 9), (3, 2), (3, 17), (13, 20), (1, 1), (9, 10), (14, 6), (2, 0), (9, 28), (8, 6), (11, 6), (3, 0), (8, 9), (14, 7), (12, 4)]
    goals = [(11, 20), (2, 19), (6, 5), (4, 18), (9, 20), (7, 0), (2, 25), (12, 4), (3, 25), (0, 12), (4, 23), (2, 24), (9, 2), (1, 6), (13, 29), (14, 29), (2, 28), (14, 16), (13, 0), (1, 27), (14, 25), (10, 20), (12, 28), (2, 29), (1, 29)]
    lengths = [36, 30, 30, 21, 28, 24, 32, 27, 42, 23, 35, 37, 23, 26, 40, 36, 42, 28, 44, 36, 38, 29, 33, 42, 44]

    if (task is None) or not (0 <= task < 25):
        task = random.randint(0, 24)

    start = Node(*starts[task])
    goal = Node(*goals[task])
    length = lengths[task]
    #try:
    if 1 != 2:
        result = search_func(task_map, start.i, start.j, goal.i, goal.j, *args)
        number_of_steps = result[2]
        nodes_created = result[3]
        if result[0]:
            path = uf.make_path(result[1])
            correct = int(path[1])  == int(length)
            draw(task_map, start, goal, path[0], *result[4:])
            print("Path found! Length: " + str(path[1]) + \
                ". Nodes created: " + str(nodes_created) + \
                ". Number of steps: " + str(number_of_steps) + ". Correct: " + str(correct))
        else:
            draw(task_map, start, goal, [start], *result[4:], *result[5:])
            print("Path not found!")
        return result

    #except Exception as e:
        print("Execution error")
        print(e)


In [21]:
def massive_test(search_function, heuristic_function, task_map, tasks, w=1, EPS=1e-6):
    corrected = 0
    results = np.zeros((3, len(tasks)))
    for i, (start_i, start_j, goal_i, goal_j, exp_len) in tqdm(enumerate(tasks)):
        try:
            result = search_function(task_map, start_i, start_j, goal_i, goal_j, heuristic_function, w)
            nodes_expanded = result[2]
            nodes_opened = result[3]
            if result[0]:
                path = uf.make_path(result[1])
                correct = abs(path[1] - exp_len) < EPS
                if correct:
                    corrected += 1
                d = path[1]
                if (exp_len > 0):
                    d /= exp_len
                results[0, i] = len(nodes_expanded)
                results[1, i] = d
                #print("Path found! Length: " + str(path[1]) + ". Nodes created: " + str(len(nodes_opened) + len(nodes_expanded)) + ". Number of steps: " + str(len(nodes_expanded)) + ". Correct: " + str(correct))
            #
                #print("Path not found!")
        except Exception as e:
            print("Execution error")
            print(e)

    print(f'Correct: {corrected} of {len(tasks)}')
    return results

In [35]:
def moscow_2_test(heuristic_func):
    map_path = "test/data/Moscow_2_1024.map"
    tasks_path = "test/data/Moscow_2_1024.map.scen"
    task_map = mutil.read_map_from_movingai_file(map_path)
    tasks = mutil.read_tasks_from_movingai_file(tasks_path)
    random.seed(50)
    random.shuffle(tasks)

    df = pd.DataFrame(columns=["4-astar", "32-astar", "theta-star"])
    for i in tqdm(range(5)):
        (s_i, s_j, f_i, f_j, l) = tasks[i]
        start = Node(i = s_i, j = s_j)
        goal = Node(i = f_i, j = f_j)

        result4 = astar2k(task_map, start.i, start.j, goal.i, goal.j, heuristic_func, Open, Closed, k=2)
        result32 = astar2k(task_map, start.i, start.j, goal.i, goal.j, heuristic_func, Open, Closed, k=5)
        resulttheta = thetastar(task_map, start.i, start.j, goal.i, goal.j, heuristic_func, Open, Closed)
        #number_of_steps = result[2]
        #nodes_created = result[3]
        l4 = -1
        l32 = -1
        ltheta = -1
        if result4[0]:
            _, l4 = uf.make_path(result4[1])
        if result32[0]:
            _, l32 = uf.make_path(result32[1])
        if resulttheta[0]:
            _, ltheta = uf.make_path(resulttheta[1])
        df = df.append({"4-astar": l4, "32-astar": l32, "theta-star": ltheta}, ignore_index=True)
    display(df)

In [36]:
moscow_2_test(uf.euclidian_distance)

100%|██████████| 5/5 [02:44<00:00, 32.82s/it]


,4-astar,32-astar,theta-star
0,708.0,609.986486,603.099633
1,1038.0,822.333673,822.879518
2,1851.0,1506.729425,1504.290037
3,1105.0,834.153699,830.628261
4,137.0,98.338150,98.334456


In [ ]:
#%time res = simple_test(astar2k, 0, euclidian_distance, Open, Closed, 5)

In [ ]:
#%time res = simple_test(thetastar, 0, euclidian_distance, YourOpen, YourClosed)

In [ ]:
for i in range(20):
    #simple_test(astar2k, i, uf.euclidian_distance, Open, Closed, 5)
    #simple_test(thetastar, i, uf.euclidian_distance, Open, Closed)
    pass

In [484]:
"""def astar_reexpansion(grid_map, start_i, start_j, goal_i, goal_j, heuristic_func = None, open_and_closed_type = YourOpenAndClosed):
    start_node = Node(i = start_i, j = start_j)
    OPEN_AND_CLOSED = open_and_closed_type()
    steps = 0
    nodes_created = 0

    OPEN_AND_CLOSED.add_node(start_node)
    while not OPEN_AND_CLOSED.is_empty():
        current = OPEN_AND_CLOSED.get_best_node()
        steps += 1
        if (current.i == goal_i and current.j == goal_j):
            return (True, current, steps, nodes_created, OPEN_AND_CLOSED, OPEN_AND_CLOSED.expanded, OPEN_AND_CLOSED.reexpanded)
        for (neighbour_i, neighbour_j) in grid_map.get_neighbors(current.i, current.j):
            next_node = Node(i = neighbour_i, j = neighbour_j, g = current.g + compute_cost(current.i, current.j, neighbour_i, neighbour_j), h = heuristic_func(neighbour_i, neighbour_j, goal_i, goal_j), parent = current, k = nodes_created)
            nodes_created += 1
            OPEN_AND_CLOSED.add_node(next_node)

    return (False, None, steps, nodes_created, OPEN_AND_CLOSED, OPEN_AND_CLOSED.expanded, OPEN_AND_CLOSED.reexpanded)
"""

In [ ]:
for i in range(20):
    #res = simple_test(astar_reexpansion, i, euclidian_distance, YourOpenAndClosed)
    #print("Number of reexpansions:", res[4].number_of_reexpansions)
    pass